To generate template for new proposals.

In [1]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

In [4]:
# Initialize the require parameters.
# file name does not require file extension. File will be saved to Downloads folder.
# system_names cannot exceed the character length of 32 as this is excel's sheet names limit.
# system_names should be put in required order.
file_name = 'J12476 MODEC - 4103 PROJECT GREEN FPSO - COM R1'
system_names = ['GMDSS SYSTEM', 
                'OPERATION SUPPORT CONSOLE', 
                'UHF SYSTEM', 
                'VHF SYSTEM', 
                'CRANE RADIOS', 
                'MASTER CLOCK, TMS & OTHERS',
                'DISPATCHER SYSTEM',
                'START-UP & COMMISSIONING SPARE', 
                'TWO YEARS SPARES',
                'ENGINEERING SERVICES']
project_info = {'Attend to: ' : "PHOEBE YANG",
                'Designation: ' : "PROCUREMENT SPECALIST",
                'Client Name: ' : "MODEC",
                'Client RFQ No: ' : "0010000476",
                'Ref Doc No: ' : '4103-MI20-55PO-6300-001, REV. B',
                'Project Name: ' : 'OB03 - UARU FPSO - 55-6300 TELECOMMUNICATION SYSTEM',
                'Prepared By: ' : "THIHA AUNG",
                'Sales Manager: ' : "SHAUN TEO",
                'Jason Ref: ' : 'J12476',
                'Revision Num: ' : 'R0',
                'Comn Site: ' : 'CHINA'}
quoted_currency = 'USD'
give_discount = True

In [5]:
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
file_path = Path(downloads_folder, file_name)
# Download template file from the internet and write to local folder
url = "https://filedn.com/liTeg81ShEXugARC7cg981h/Proposal_Template.xlsx"
resp = requests.get(url)

with open("Template.xlsx", 'wb') as fd:
    for chunk in resp.iter_content(chunk_size=8192):
        fd.write(chunk)
system_names = [x.upper() for x in system_names]
# Copy sheet from template to new workbook
new_book = xw.Book()
template = xw.Book("Template.xlsx", password=hide.new)
template.sheets['config'].copy(after=new_book.sheets[0])
new_book.sheets['Sheet1'].delete()
template.sheets['Cover'].copy(after=new_book.sheets['config'])

# Project info
# Set exchange rates
sheet = new_book.sheets['Config']
# Quoted currency
sheet.range('B12').value = quoted_currency
# Project info
sheet.range('B21').value = project_info['Attend to: ']
sheet.range('B22').value = project_info['Designation: ']
sheet.range('B23').value = project_info['Client Name: ']
sheet.range('B24').value = project_info['Client RFQ No: ']
sheet.range('B25').value = project_info['Ref Doc No: ']
sheet.range('B26').value = project_info['Project Name: ']
sheet.range('B27').value = project_info['Prepared By: ']
sheet.range('B28').value = project_info['Sales Manager: ']
sheet.range('B29').value = project_info['Jason Ref: ']
sheet.range('B30').value = project_info['Revision Num: ']
# sheet.range('B31').value = project_info['']
sheet.range('B32').value = datetime.today().strftime('%Y-%m-%d')

# Set up formula in Cover sheet
sheet = new_book.sheets['Cover']
sheet.range('D7').formula = '=Config!B26'
sheet.range('C42').formula = '=Config!B21'
sheet.range('C43').formula = '=Config!B23'
sheet.range('C44').formula = '=Config!B24'
sheet.range('C45').formula = '=Config!B29'
sheet.range('C46').formula = '=Config!B30'
sheet.range('C47').formula = '=Config!B32'
sheet.range('D39').formula = '=Config!B13'

for system in system_names[::-1]:
    sheet_name = 'Cover'
    template.sheets['System'].copy(after=new_book.sheets[sheet_name])
    sheet_name = system
    new_book.sheets['System'].name = sheet_name
    # Set formula to reference Config.
    new_book.sheets[sheet].range('A1').formula = '= "JASON REF: " & Config!B29 &  ", REVISION: " &  Config!B30 & ", PROJECT: " & Config!B26'
template.sheets['Summary'].copy(after=new_book.sheets['Cover'])
template.sheets['Technical_Notes'].copy(after=new_book.sheets[-1])
template.sheets['T&C'].copy(after=new_book.sheets[-1])
for sheet in new_book.sheet_names:
    if sheet in ['Summary','Technical_Notes', 'T&C']:
        new_book.sheets[sheet].range('A1').formula = '= "JASON REF: " & Config!B29 &  ", REVISION: " &  Config!B30 & ", PROJECT: " & Config!B26'

# Set default for engineering services
sheet = new_book.sheets['ENGINEERING SERVICES']
sheet.range('J1').value = 0.3
sheet.range('L1').value = 0
sheet.range('N1').value = 0
sheet.range('P1').value = 0
sheet.range('R1').value = 0

template.close()

ValueError: The max. length of a sheet name is 31 characters. Yours is 47.

In [11]:
# Write necessary formula to excel
for system in system_names:
    sheet = new_book.sheets[system]
    functions.fill_formula(sheet)
    functions.format(sheet)

In [12]:
functions.fill_lastrow(new_book)

In [13]:
functions.summary(new_book)

In [14]:
# Format
for system in system_names:
    sheet = new_book.sheets[system]
    functions.format(sheet)

In [15]:
# If the book with the same name already open, it will cuase error.
new_book.sheets['Summary'].activate()
new_book.save(downloads_folder + '/' + file_name + '.xlsx', password=hide.legacy) 
# new_book.close()